In [1]:
import PIL
import cv2
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

import os
import random

from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
main_dir = '/kaggle/input/garbage-classification/garbage_classification/'

In [9]:
# reading the file
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.1,
    validation_split=0.15
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15
)

train_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    seed=42
)

validation_generator = test_datagen.flow_from_directory(
    main_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    seed=42
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 13194 images belonging to 12 classes.
Found 2321 images belonging to 12 classes.
{0: 'battery', 1: 'biological', 2: 'brown-glass', 3: 'cardboard', 4: 'clothes', 5: 'green-glass', 6: 'metal', 7: 'paper', 8: 'plastic', 9: 'shoes', 10: 'trash', 11: 'white-glass'}


In [4]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [11]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(224, 224, 3)),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(12, activation='softmax')
])

model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 12)                15372     
                                                                 
Total params: 2273356 (8.67 MB)
Trainable params: 15372 (60.05 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=2,
                                                  mode ="max",
                                                  verbose=1,
                                                  min_delta=0.001,
                                                  restore_best_weights=True)

In [13]:
tf.keras.utils.set_random_seed(67)
tf.config.experimental.enable_op_determinism()
history = model.fit(train_generator,
                   epochs=5,
                   validation_data=validation_generator,
                   callbacks=[early_stopping])

Epoch 1/5
207/207 [==============================] - 193s 920ms/step - loss: 0.6022 - accuracy: 0.8229 - val_loss: 0.4021 - val_accuracy: 0.8699
Epoch 2/5
207/207 [==============================] - 188s 910ms/step - loss: 0.2551 - accuracy: 0.9255 - val_loss: 0.3215 - val_accuracy: 0.8962
Epoch 3/5
207/207 [==============================] - 188s 910ms/step - loss: 0.1983 - accuracy: 0.9416 - val_loss: 0.3006 - val_accuracy: 0.9009
Epoch 4/5
207/207 [==============================] - 190s 917ms/step - loss: 0.1670 - accuracy: 0.9509 - val_loss: 0.2910 - val_accuracy: 0.9018
Epoch 5/5
207/207 [==============================] - 189s 914ms/step - loss: 0.1447 - accuracy: 0.9574 - val_loss: 0.2782 - val_accuracy: 0.9104


In [14]:
model.save('mobilenetv2_5epochs.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
